In [ ]:
# Location
#   cluster together similar locations
#   use google maps api

# status
#   single = available, married, seeing someone

# orientation - ok

# body type - make it ordinal
#   fit = athletic, jacked, overweight, curvy = a little extra = full figured, skinny = thin, drop used up, rather not say --> null

# diet
#   merge all anything, merge all vegetarian, merge all vegan, merge all halal, merge all kosher, merge all other

# drinks - ordinal, null --> socially (mode)

# drugs - OK

# education
#  dropped out -> merge with level below that, hs, college, (merge masters, phd, law, med), space camp (merge graduated and working on)

# ethnicity
# white, black, asian = pacific islander, hispanic / latin, native american, middle eastern, mixed, other

# age, height
#   drop nulls

# income - drop

# job
#  change rather not say --> null

# drop last online

# offspring
#  convert into 2 columns - has kids, wants kids and null --> 0, 0

# pets
#   convert into 4 columns has dogs, likes dogs, has cats, likes cats, null --> all zeros

# religion
#   nan - null, one hot encoding for each religion, ignore seriousness

# sign
#   one hot encoding for different compatible groups, how serious they are about it single column

# smokes - OK

# speaks
#   one hot encode each major language, everything else is another category -> other and don't count poorly

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)

df = pd.read_csv('/content/drive/My Drive/CS 5112 Final Project/Code/okcupid_profiles.csv')

Mounted at /content/drive


Andrew:  

assuming dataset is named: df

features needs one-hot-encoding: [ job, ethnicity ]

In [3]:
#Height & age
df['height'].replace(0, None, inplace = True)
#for x in ['age', 'height']:
  #df[x] = 2* (df[x] - df[x].min())/ (df[x].max() - df[x].min())
# ethnicity
# white, black, asian = pacific islander, hispanic / latin, native american, middle eastern, mixed, other
ethnicity_l = df['ethnicity'].tolist()
new_l = []
for e in ethnicity_l:
  if e is np.nan or e is None:
    new_l.append(None)
    continue
  # print(e)  
  arr = e.split(',')
  if len(arr) == 1:
    if e == 'pacific islander':
      new_l.append('asian')
    elif e == 'other':
      new_l.append('mixed')
    else:
      new_l.append(e)
  else:
    new_l.append('mixed')

df['ethnicity'] = new_l

#job
df['job'].replace('rather not say', None, inplace=True)
display(df['job'].unique())

array(['transportation', 'hospitality / travel', nan, 'student',
       'artistic / musical / writer', 'computer / hardware / software',
       'banking / financial / real estate', 'entertainment / media',
       'sales / marketing / biz dev', 'other', 'medicine / health',
       'science / tech / engineering', 'executive / management',
       'education / academia', 'clerical / administrative',
       'construction / craftsmanship', 'political / government',
       'law / legal services', 'unemployed', 'military', 'retired'],
      dtype=object)

Preprocessing 'speaks'

In [4]:
def strip_l(input):
  arr = input.split(',')
  new_l = []
  ### optional 12 most common languages
  common_l = {'english','italian','arabic','chinese','mandarin','spanish','hindi','portuguese','russian','japanese','german','korean','turkish','telugu','marathi'}
  for x in arr:
    temp_arr = x.strip().split(' ')
    if len(temp_arr) >1 and temp_arr[1] == '(poorly)':
      continue
      ###optional 12 most common languages
    if temp_arr[0] in common_l:
      new_l.append(temp_arr[0])
  s = ','.join(new_l)
  return s

strip_l('english (fluently), italian (okay), chinese (poorly)')

'english,italian'

In [5]:
lang_l = df['speaks'].tolist()
lan_new = []

for l in lang_l:
  if l is np.nan or l is None:
    lan_new.append(None)
    continue
  lan_new.append(strip_l(l))

df['speaks']= lan_new
temp = df['speaks'].str.get_dummies(sep=',')
df.drop(columns =['speaks'], inplace=True)
df[temp.columns]= temp

In [6]:
df['arabic'] = df['arabic'].apply(lambda x: 'yes' if x == 1 else 'no')
df['chinese'] = df['chinese'].apply(lambda x: 'yes' if x == 1 else 'no')
df['english'] = df['english'].apply(lambda x: 'yes' if x == 1 else 'no')
df['german'] = df['german'].apply(lambda x: 'yes' if x == 1 else 'no')
df['hindi'] = df['hindi'].apply(lambda x: 'yes' if x == 1 else 'no')
df['italian'] = df['italian'].apply(lambda x: 'yes' if x == 1 else 'no')
df['japanese'] = df['japanese'].apply(lambda x: 'yes' if x == 1 else 'no')
df['korean'] = df['korean'].apply(lambda x: 'yes' if x == 1 else 'no')
df['portuguese'] = df['portuguese'].apply(lambda x: 'yes' if x == 1 else 'no')
df['russian'] = df['russian'].apply(lambda x: 'yes' if x == 1 else 'no')
df['spanish'] = df['spanish'].apply(lambda x: 'yes' if x == 1 else 'no')
df['turkish'] = df['turkish'].apply(lambda x: 'yes' if x == 1 else 'no')

Kristjan:

Creates one hot encoded columns for offsprings, pets and religion

In [7]:
# offspring
# convert into 2 columns - has kids, wants kids and null --> 0, 0
off = pd.get_dummies(df['offspring'])

new_offsprings = pd.DataFrame(columns=['has kids', 'wants kids'])

# # print(off.shape)

for index, row in df.iterrows():
    has = 'no'
    wants = 'no'
    if off["has a kid"][index] == 1 or off["has a kid, and might want more"][index] == 1 or off["has a kid, and wants more"][index] == 1 or off["has a kid, but doesn't want more"][index] == 1 or off["has kids"][index] == 1 or off["has kids, and might want more"][index] == 1 or off["has kids, and wants more"][index] == 1 or off["has kids, but doesn't want more"][index] == 1:
      has = 'yes'
    if off["doesn't have kids, but might want them"][index] == 1 or off["doesn't have kids, but wants them"][index] == 1 or off["has a kid, and might want more"][index] == 1 or off["has a kid, and wants more"][index] == 1 or off["has kids, and might want more"][index] == 1 or off["has kids, and wants more"][index] == 1 or off["might want kids"][index] == 1 or off["wants kids"][index] == 1:
      wants = 'yes'
    new_offsprings = new_offsprings.append({'has kids': has, 'wants kids': wants}, ignore_index=True)

# new_offsprings

# # pets

pets = pd.get_dummies(df['pets'])

new_pets = pd.DataFrame(columns=['likes dogs', 'likes cats'])

for index, row in df.iterrows():
    dogs = 'no'
    cats = 'no'
    if pets["has dogs"][index] == 1 or pets["has dogs and dislikes cats"][index] == 1 or pets["has dogs and has cats"][index] == 1 or pets["has dogs and likes cats"][index] == 1 or pets["likes dogs"][index] == 1 or pets["likes dogs and dislikes cats"][index] == 1 or pets["likes dogs and has cats"][index] == 1 or pets["likes dogs and likes cats"][index] == 1:
      dogs = 'yes'
    if pets["dislikes dogs and has cats"][index] == 1 or pets["dislikes dogs and likes cats"][index] == 1 or pets["has cats"][index] == 1 or pets["has dogs and has cats"][index] == 1 or pets["has dogs and likes cats"][index] == 1 or pets["likes cats"][index] == 1 or pets["likes dogs and has cats"][index] == 1 or pets["likes dogs and likes cats"][index] == 1:
      cats = 'yes'
    new_pets = new_pets.append({'likes dogs': dogs, 'likes cats': cats}, ignore_index=True)

# new_pets

In [8]:
# religion

rel = pd.get_dummies(df['religion'])

new_religion = pd.DataFrame(columns=['agnosticism', 'atheism', 'buddhism', 'catholicism', 'christianity', 'hinduism', 'islam', 'judaism', 'other'])

for index, row in df.iterrows():
    agnosticism = 0
    atheism = 0
    buddhism = 0
    catholicism = 0
    christianity = 0
    hinduism = 0
    islam = 0
    judaism = 0
    other = 0
    if rel["agnosticism"][index] == 1 or rel["agnosticism and laughing about it"][index] == 1 or rel["agnosticism and somewhat serious about it"][index] == 1 or rel["agnosticism and very serious about it"][index] == 1 or rel["agnosticism but not too serious about it"][index] == 1:
      agnosticism = 1
    if rel["atheism"][index] == 1 or rel["atheism and laughing about it"][index] == 1 or rel["atheism and somewhat serious about it"][index] == 1 or rel["atheism and very serious about it"][index] == 1 or rel["atheism but not too serious about it"][index] == 1:
      atheism = 1
    if rel["buddhism"][index] == 1 or rel["buddhism and laughing about it"][index] == 1 or rel["buddhism and somewhat serious about it"][index] == 1 or rel["buddhism and very serious about it"][index] == 1 or rel["buddhism but not too serious about it"][index] == 1:
      buddhism = 1
    if rel["catholicism"][index] == 1 or rel["catholicism and laughing about it"][index] == 1 or rel["catholicism and somewhat serious about it"][index] == 1 or rel["catholicism and very serious about it"][index] == 1 or rel["catholicism but not too serious about it"][index] == 1:
      catholicism = 1
    if rel["christianity"][index] == 1 or rel["christianity and laughing about it"][index] == 1 or rel["christianity and somewhat serious about it"][index] == 1 or rel["christianity and very serious about it"][index] == 1 or rel["christianity but not too serious about it"][index] == 1:
      christianity = 1
    if rel["hinduism"][index] == 1 or rel["hinduism and laughing about it"][index] == 1 or rel["hinduism and somewhat serious about it"][index] == 1 or rel["hinduism and very serious about it"][index] == 1 or rel["hinduism but not too serious about it"][index] == 1:
      hinduism = 1
    if rel["islam"][index] == 1 or rel["islam and laughing about it"][index] == 1 or rel["islam and somewhat serious about it"][index] == 1 or rel["islam and very serious about it"][index] == 1 or rel["islam but not too serious about it"][index] == 1:
      islam = 1
    if rel["judaism"][index] == 1 or rel["judaism and laughing about it"][index] == 1 or rel["judaism and somewhat serious about it"][index] == 1 or rel["judaism and very serious about it"][index] == 1 or rel["judaism but not too serious about it"][index] == 1:
      judaism = 1
    if rel["other"][index] == 1 or rel["other and laughing about it"][index] == 1 or rel["other and somewhat serious about it"][index] == 1 or rel["other and very serious about it"][index] == 1 or rel["other but not too serious about it"][index] == 1:
      other = 1
    
    new_religion = new_religion.append({'agnosticism': agnosticism, 'atheism': atheism, 'buddhism': buddhism, 'catholicism': catholicism, 'christianity': christianity, 'hinduism': hinduism, 'islam': islam, 'judaism': judaism, 'other': other}, ignore_index=True)

# new_religion

In [9]:
new_religion = new_religion.astype(int)

In [10]:
new_religion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59946 entries, 0 to 59945
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   agnosticism   59946 non-null  int64
 1   atheism       59946 non-null  int64
 2   buddhism      59946 non-null  int64
 3   catholicism   59946 non-null  int64
 4   christianity  59946 non-null  int64
 5   hinduism      59946 non-null  int64
 6   islam         59946 non-null  int64
 7   judaism       59946 non-null  int64
 8   other         59946 non-null  int64
dtypes: int64(9)
memory usage: 4.1 MB


In [11]:
new_religion

,agnosticism,atheism,buddhism,catholicism,christianity,hinduism,islam,judaism,other
0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
59941,0,0,0,1,0,0,0,0,0
59942,1,0,0,0,0,0,0,0,0
59943,0,0,0,0,1,0,0,0,0
59944,1,0,0,0,0,0,0,0,0


In [12]:
religion = new_religion.copy()

In [13]:
def get_rel(row):
  if row['agnosticism'] == 1 :
    return 'agnosticism'
  if row['atheism'] == 1 :
    return 'atheism'
  if row['buddhism'] == 1 :
    return 'buddhism'
  if row['catholicism'] == 1:
    return 'catholicism'
  if row['christianity']  == 1:
    return 'christianity'
  if row['hinduism'] == 1:   
    return 'hinduism.'
  if row['islam'] == 1:
    return 'islam'
  if row['judaism'] == 1:
    return 'judaism'
  if row['other'] == 1:
    return 'other'
  else:
    return 'none'

In [14]:
religion['new_religion'] = religion.apply(lambda row: get_rel(row), axis=1)

In [15]:
religion['new_religion'].value_counts()

none            20226
agnosticism      8812
other            7743
atheism          6985
christianity     5787
catholicism      4758
judaism          3098
buddhism         1948
hinduism.         450
islam             139
Name: new_religion, dtype: int64

In [16]:
processed_data = pd.concat([new_offsprings, new_pets, religion['new_religion']], axis=1)

In [17]:
processed_data

,has kids,wants kids,likes dogs,likes cats,new_religion
0,no,yes,yes,yes,agnosticism
1,no,yes,yes,yes,agnosticism
2,no,no,no,yes,none
3,no,no,no,yes,none
4,no,no,yes,yes,none
...,...,...,...,...,...
59941,yes,no,yes,no,catholicism
59942,no,no,yes,yes,agnosticism
59943,no,no,no,no,christianity
59944,no,yes,yes,yes,agnosticism


In [18]:
df = pd.concat([df, processed_data], axis=1)

In [19]:
df.shape

(59946, 47)

In [20]:
df.drop(columns=['pets', 'religion', 'offspring'], inplace=True)

In [21]:
df.shape

(59946, 44)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59946 entries, 0 to 59945
Data columns (total 44 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   age           59946 non-null  int64  
 1   status        59946 non-null  object 
 2   sex           59946 non-null  object 
 3   orientation   59946 non-null  object 
 4   body_type     54650 non-null  object 
 5   diet          35551 non-null  object 
 6   drinks        56961 non-null  object 
 7   drugs         45866 non-null  object 
 8   education     53318 non-null  object 
 9   ethnicity     54266 non-null  object 
 10  height        59943 non-null  float64
 11  income        59946 non-null  int64  
 12  job           51693 non-null  object 
 13  last_online   59946 non-null  object 
 14  location      59946 non-null  object 
 15  sign          48890 non-null  object 
 16  smokes        54434 non-null  object 
 17  essay0        54458 non-null  object 
 18  essay1        52374 non-nu

Anjali - cleaning up body_type, diet, drinks, education

# Cleaning body_type

In [23]:
# Our notes from above:
# body type - make it ordinal
#   fit = athletic, jacked, overweight, curvy = a little extra = full figured, skinny = thin, drop used up, rather not say --> null

# Order
# 1 -> overweight
# 2 -> curvy, a little extra, full figured
# 3 -> average
# 4 -> skinny, thin
# 5 -> athletic, fit
# 6 -> jacked

# drop used up and convert rather not say to null
df = df[df.body_type != "used up"]
df.loc[df['body_type'] == 'rather not say','body_type'] = np.nan

# making sure we did it correctly
# print(df['body_type'].value_counts())
# print(df['body_type'].isna().sum()) # includes already NaN values

# curvy = a little extra = full figured
df.loc[df['body_type'] == 'a little extra','body_type'] = "curvy"
df.loc[df['body_type'] == 'full figured','body_type'] = "curvy"

# skinny = thin
df.loc[df['body_type'] == 'skinny','body_type'] = "thin"

# athletic = fit
df.loc[df['body_type'] == 'athletic','body_type'] = "fit"

# making sure we did it correctly
# print(df['body_type'].value_counts())

# make all of them ordinal!
df.loc[df['body_type'] == 'overweight','body_type'] = 1
df.loc[df['body_type'] == 'curvy','body_type'] = 2
df.loc[df['body_type'] == 'average','body_type'] = 3
df.loc[df['body_type'] == 'thin','body_type'] = 4
df.loc[df['body_type'] == 'fit','body_type'] = 5
df.loc[df['body_type'] == 'jacked','body_type'] = 6
pd.to_numeric(df['body_type'])
# display(df['body_type'])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


0        2.0
1        3.0
2        4.0
3        4.0
4        5.0
        ... 
59941    NaN
59942    5.0
59943    3.0
59944    5.0
59945    3.0
Name: body_type, Length: 59591, dtype: float64

In [24]:
df.shape

(59591, 44)

# Cleaning diets

In [25]:
# Our notes from above:
# diet
#   merge all anything, merge all vegetarian, merge all vegan, merge all halal, merge all kosher, merge all other

arr = ["anything", "vegetarian", "vegan", "halal", "kosher", "other"]

for diet_type in arr:
    df.loc[df['diet'].str.contains(diet_type, na=False), 'diet'] = diet_type

# making sure we did it right
print(df['diet'].value_counts())
print(df['diet'].isna().sum()) # includes already NaN values

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


anything      27684
vegetarian     4963
other          1754
vegan           694
kosher          111
halal            73
Name: diet, dtype: int64
24312


In [26]:
df.shape

(59591, 44)

# Cleaning drinks

In [27]:
# Our notes from above:
# drinks - ordinal, null --> socially (mode)

print(df['drinks'].value_counts())

# Order
# not at all -> 0
# rarely -> 1
# socially -> 2
# often -> 3
# very often -> 4
# desperately -> 5

# make all of them ordinal!
df.loc[df['drinks'] == 'not at all','drinks'] = 0
df.loc[df['drinks'] == 'rarely','drinks'] = 1
df.loc[df['drinks'] == 'socially','drinks'] = 2
df.loc[df['drinks'] == 'often','drinks'] = 3
df.loc[df['drinks'] == 'very often','drinks'] = 4
df.loc[df['drinks'] == 'desperately','drinks'] = 5

# null -> mode (socially) -> 2
df["drinks"].fillna(2, inplace = True)

pd.to_numeric(df['drinks'])

# making sure we did it right
print(df['drinks'].value_counts())
print(df['drinks'].isna().sum()) # includes already NaN values

socially       41610
rarely          5938
often           5104
not at all      3244
very often       453
desperately      281
Name: drinks, dtype: int64
2    44571
1     5938
3     5104
0     3244
4      453
5      281
Name: drinks, dtype: int64
0


In [28]:
df.shape

(59591, 44)

In [29]:
df['location'].value_counts()

san francisco, california    30883
oakland, california           7153
berkeley, california          4177
san mateo, california         1328
palo alto, california         1059
                             ...  
boise, idaho                     1
santa ana, california            1
fort lauderdale, florida         1
crowley, texas                   1
campbell, california             1
Name: location, Length: 198, dtype: int64

# Cleaning education

In [30]:
# Our notes from above:
# education
#  dropped out -> merge with level below that, hs, college, (merge masters, phd, law, med), space camp (merge graduated and working on)

# see vals
print("OG!:")
print(df['education'].value_counts())

# merge dropped out with level below that
df.loc[(df['education'].str.contains('dropped')) & (df['education'].str.contains('college')), 'education'] = "graduated from high school"
df.loc[(df['education'].str.contains('dropped')) & (df['education'].str.contains('masters')), 'education'] = "graduated from college"
df.loc[(df['education'].str.contains('dropped')) & (df['education'].str.contains('ph.d')), 'education'] = "graduated from college"
df.loc[(df['education'].str.contains('dropped')) & (df['education'].str.contains('med school')), 'education'] = "graduated from college/university"
df.loc[(df['education'].str.contains('dropped')) & (df['education'].str.contains('law school')), 'education'] = "graduated from college/university"


# merge everything space camp lol, everything hs, everything college
df.loc[df['education'].str.contains("space camp", na=False), 'education'] = "space camp"

# merge working on masters, phd, law and med -> working on grad
df.loc[(df['education'].str.contains('working')) & (df['education'].str.contains('masters')), 'education'] = "working on grad"
df.loc[(df['education'].str.contains('working')) & (df['education'].str.contains('ph.d')), 'education'] = "working on grad"
df.loc[(df['education'].str.contains('working')) & (df['education'].str.contains('law school')), 'education'] = "working on grad"
df.loc[(df['education'].str.contains('working')) & (df['education'].str.contains('med school')), 'education'] = "working on grad"

# two year = regular college
df.loc[(df['education'].str.contains('working')) & (df['education'].str.contains('two-year college')), 'education'] = "working on college/university"
df.loc[(df['education'].str.contains('graduated')) & (df['education'].str.contains('two-year college')), 'education'] = "graduated from college/university"

# merge graduated from masters, phd, law and med -> graduated from grad
df.loc[(df['education'].str.contains('graduated')) & (df['education'].str.contains('masters')), 'education'] = "graduated from grad"
df.loc[(df['education'].str.contains('graduated')) & (df['education'].str.contains('ph.d')), 'education'] = "graduated from grad"
df.loc[(df['education'].str.contains('graduated')) & (df['education'].str.contains('law school')), 'education'] = "graduated from grad"
df.loc[(df['education'].str.contains('graduated')) & (df['education'].str.contains('med school')), 'education'] = "graduated from grad"

# college = two-year college = working on college/university
df.loc[df['education'] == 'college/university','education'] = "working on college/university"
df.loc[df['education'] == 'two-year college','education'] = "working on college/university"

# high school = working on high school
df.loc[df['education'] == 'high school','education'] = "working on high school"

# law, ph.d, masters = working on grad
df.loc[df['education'] == 'masters program','education'] = "working on grad"
df.loc[df['education'] == 'law school','education'] = "working on grad"
df.loc[df['education'] == 'med school','education'] = "working on grad"
df.loc[df['education'] == 'ph.d program','education'] = "working on grad"

# see vals
print(" ")
print("NEW!:")
print(df['education'].value_counts())
display(df)

OG!:
graduated from college/university    23842
graduated from masters program        8944
working on college/university         5677
working on masters program            1671
graduated from two-year college       1528
graduated from high school            1424
graduated from ph.d program           1264
graduated from law school             1119
working on two-year college           1069
dropped out of college/university      986
working on ph.d program                979
college/university                     795
graduated from space camp              637
dropped out of space camp              483
graduated from med school              445
working on space camp                  430
working on law school                  265
two-year college                       222
working on med school                  210
dropped out of two-year college        187
dropped out of masters program         138
masters program                        136
dropped out of ph.d program            127
droppe

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,height,income,job,last_online,location,sign,smokes,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9,arabic,chinese,english,german,hindi,italian,japanese,korean,portuguese,russian,spanish,turkish,has kids,wants kids,likes dogs,likes cats,new_religion
0,22,single,m,straight,2,anything,2,never,working on college/university,mixed,75.0,-1,transportation,2012-06-28-20-30,"south san francisco, california",gemini,sometimes,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...,no,no,yes,no,no,no,no,no,no,no,no,no,no,yes,yes,yes,agnosticism
1,35,single,m,straight,3,other,3,sometimes,space camp,white,70.0,80000,hospitality / travel,2012-06-29-21-41,"oakland, california",cancer,no,i am a chef: this is what that means. 1. i am ...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories. my b...,NaN,NaN,i am very open and will share just about anyth...,NaN,no,no,yes,no,no,no,no,no,no,no,no,no,no,yes,yes,yes,agnosticism
2,38,available,m,straight,4,anything,2,NaN,graduated from grad,None,68.0,-1,NaN,2012-06-27-09-10,"san francisco, california",pisces but it doesn&rsquo;t matter,no,"i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement conversation creation contemplation t...,NaN,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ...",no,no,yes,no,no,no,no,no,no,no,no,no,no,no,no,yes,none
3,23,single,m,straight,4,vegetarian,2,NaN,working on college/university,white,71.0,20000,student,2012-06-28-14-22,"berkeley, california",pisces,no,i work in a library and go to school. . .,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,"bataille, celine, beckett. . . lynch, jarmusch...",NaN,cats and german philosophy,NaN,NaN,you feel so inclined.,no,no,yes,no,no,no,no,no,no,no,no,no,no,no,no,yes,none
4,29,single,m,straight,5,NaN,2,never,graduated from college/university,mixed,66.0,-1,artistic / musical / writer,2012-06-27-21-26,"san francisco, california",aquarius,no,hey how's it going? currently vague on the pro...,work work work work + play,creating imagery to look at: http://bagsbrown....,i smile a lot and my inquisitive nature,"music: bands, rappers, musicians at the moment...",NaN,NaN,NaN,NaN,NaN,no,no,yes,no,no,no,no,no,no,no,no,no,no,no,yes,yes,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59941,59,single,f,straight,NaN,NaN,2,never,graduated from college/university,None,62.0,-1,sales / marketing / biz dev,2012-06-12-21-47,"oakland, california",cancer and it&rsquo;s fun to think about,no,"vibrant, expressive, caring optimist. i love b...",the happiest times have been when life came to...,i make an outstanding osso bucco. i am also ve...,"i am told that people notice my smile, eyes an...",i am an avid movie watcher and follow the broa...,"my family, my dog, italy, words and music!",writing my book.,"running with my dog, finishing up the work wee...",i have a dream to sing at the alconquin in nyc...,you ar

In [31]:
df['location'].value_counts()

san francisco, california    30883
oakland, california           7153
berkeley, california          4177
san mateo, california         1328
palo alto, california         1059
                             ...  
boise, idaho                     1
santa ana, california            1
fort lauderdale, florida         1
crowley, texas                   1
campbell, california             1
Name: location, Length: 198, dtype: int64

# Clustering Locations

In [32]:
df = df[df.location.str.contains("california", na=False)]

In [33]:
df.shape

(59502, 44)

In [34]:
locs = pd.read_csv('/content/drive/My Drive/CS 5112 Final Project/Code/locations_cali_clusters_4.csv')
locs_dict = dict(zip(locs.location, locs.cluster))

In [35]:
df['location'] = df['location'].replace(locs_dict)

In [36]:
df['location'].value_counts()

0    59431
1       44
2       20
3        7
Name: location, dtype: int64

In [37]:
df['location'] = df['location'].replace({0: 'A', 1: 'B', 2: 'C', 3: 'D'})

In [38]:
df = df[df['location'] == 'A']

In [39]:
df['location'].value_counts()

A    59431
Name: location, dtype: int64

In [40]:
df['status'] = df['status'].replace({'available': 'single'})

In [41]:
df['sign'] = df['sign'].astype(str)

In [42]:
def sign(x):
  if 'matters a lot' in x:
    return 'matters'
  elif 'doesn&rsquo;t matter' in x:
   return 'does_not_matter'
  else:
    return 'whatever'
#'matters' if 'matters a lot' in x else ('not_matter' if 'doesn&rsquo;t matter' in x else 'whatever')

In [43]:
df['sign'] = df['sign'].apply(lambda x: sign(x))

In [44]:
df['sign'].value_counts()

whatever           42161
does_not_matter    16613
matters              657
Name: sign, dtype: int64

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59431 entries, 0 to 59945
Data columns (total 44 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   age           59431 non-null  int64  
 1   status        59431 non-null  object 
 2   sex           59431 non-null  object 
 3   orientation   59431 non-null  object 
 4   body_type     53950 non-null  object 
 5   diet          35181 non-null  object 
 6   drinks        59431 non-null  int64  
 7   drugs         45487 non-null  object 
 8   education     52857 non-null  object 
 9   ethnicity     53822 non-null  object 
 10  height        59429 non-null  float64
 11  income        59431 non-null  int64  
 12  job           51257 non-null  object 
 13  last_online   59431 non-null  object 
 14  location      59431 non-null  object 
 15  sign          59431 non-null  object 
 16  smokes        53975 non-null  object 
 17  essay0        53985 non-null  object 
 18  essay1        51913 non-nu

In [46]:
cols = [col for col in df.columns if 'essay' not in col]

In [47]:
len(cols)

34

In [48]:
df = df[cols]

In [49]:
df.shape

(59431, 34)

In [50]:
df.drop(columns=['income', 'last_online'], inplace=True)
df

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,height,job,location,sign,smokes,arabic,chinese,english,german,hindi,italian,japanese,korean,portuguese,russian,spanish,turkish,has kids,wants kids,likes dogs,likes cats,new_religion
0,22,single,m,straight,2,anything,2,never,working on college/university,mixed,75.0,transportation,A,whatever,sometimes,no,no,yes,no,no,no,no,no,no,no,no,no,no,yes,yes,yes,agnosticism
1,35,single,m,straight,3,other,3,sometimes,space camp,white,70.0,hospitality / travel,A,whatever,no,no,no,yes,no,no,no,no,no,no,no,no,no,no,yes,yes,yes,agnosticism
2,38,single,m,straight,4,anything,2,NaN,graduated from grad,None,68.0,NaN,A,does_not_matter,no,no,no,yes,no,no,no,no,no,no,no,no,no,no,no,no,yes,none
3,23,single,m,straight,4,vegetarian,2,NaN,working on college/university,white,71.0,student,A,whatever,no,no,no,yes,no,no,no,no,no,no,no,no,no,no,no,no,yes,none
4,29,single,m,straight,5,NaN,2,never,graduated from college/university,mixed,66.0,artistic / musical / writer,A,whatever,no,no,no,yes,no,no,no,no,no,no,no,no,no,no,no,yes,yes,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59941,59,single,f,straight,NaN,NaN,2,never,graduated from college/university,None,62.0,sales / marketing / biz dev,A,whatever,no,no,no,yes,no,no,no,no,no,no,no,no,no,yes,no,yes,no,catholicism
59942,24,single,m,straight,5,anything,3,sometimes,working on college/university,mixed,72.0,entertainment / media,A,does_not_matter,no,no,no,yes,no,no,no,no,no,no,no,no,no,no,no,yes,yes,agnosticism
59943,42,single,m,straight,3,anything,0,never,graduated from grad,asian,71.0,construction / craftsmanship,A,does_not_matter,no,no,no,yes,no,no,no,no,no,no,no,no,no,no,no,no,no,christianity
59944,27,single,m,straight,5,anything,2,often,working on college/university,mixed,73.0,medicine / health,A,whatever,trying to quit,no,no,yes,no,no,no,no,no,no,no,no,no,no,yes,yes,yes,agnosticism


In [51]:
df.shape

(59431, 32)

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59431 entries, 0 to 59945
Data columns (total 32 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   age           59431 non-null  int64  
 1   status        59431 non-null  object 
 2   sex           59431 non-null  object 
 3   orientation   59431 non-null  object 
 4   body_type     53950 non-null  object 
 5   diet          35181 non-null  object 
 6   drinks        59431 non-null  int64  
 7   drugs         45487 non-null  object 
 8   education     52857 non-null  object 
 9   ethnicity     53822 non-null  object 
 10  height        59429 non-null  float64
 11  job           51257 non-null  object 
 12  location      59431 non-null  object 
 13  sign          59431 non-null  object 
 14  smokes        53975 non-null  object 
 15  arabic        59431 non-null  object 
 16  chinese       59431 non-null  object 
 17  english       59431 non-null  object 
 18  german        59431 non-nu

In [55]:
#df_big = df.copy()

In [56]:
# df_big['diet'].fillna('none', inplace=True)
# df_big['drugs'].fillna('none', inplace=True)
# df_big['education'].fillna('none', inplace=True)
# df_big['ethnicity'].fillna('none', inplace=True)
# df_big['job'].fillna('none', inplace=True)
# df_big['smokes'].fillna('none', inplace=True)

In [ ]:
#df_big.to_csv('/content/drive/My Drive/CS 5112 Final Project/Code/okcupid-kproto-all.csv')

In [ ]:
#new_df_big = pd.read_csv('/content/drive/My Drive/CS 5112 Final Project/Code/okcupid-kproto-all.csv')

In [57]:
# df_sample_big = df_big.sample(n=10000, random_state=1)

In [59]:
# df_sample_big.drop(columns=['location'], inplace=True)

In [ ]:
# df_sample_big.to_csv('/content/drive/My Drive/CS 5112 Final Project/Code/okcupid-kproto-all-10k.csv')

In [53]:
final_df = df.copy()
final_df.dropna(inplace=True)

In [54]:
final_df.shape

(20794, 32)

In [64]:
final_df.drop(columns=['location'], inplace=True)

In [55]:
#to_ohe = ['status', 'sex', 'orientation', 'diet', 'drugs', 'education', 'ethnicity', 'job', 'location', 'sign', 'smokes']

In [56]:
#final_df = pd.get_dummies(df, dummy_na=True, columns=to_ohe, drop_first=True)

In [57]:
#final_df.to_csv('/content/drive/My Drive/CS 5112 Final Project/Code/okcupid-kproto-20k.csv')

In [ ]:
#final_df_keep = pd.get_dummies(df, dummy_na=True, columns=to_ohe, drop_first=False)
#final_df_keep.to_csv('/content/drive/My Drive/CS 5112 Final Project/Code/okcupid-cleaned-keepall.csv') 

In [58]:
#new_df = pd.read_csv('/content/drive/My Drive/CS 5112 Final Project/Code/okcupid-kproto-20k.csv')

In [65]:
df_sample = final_df.sample(n=10000, random_state=1)

In [66]:
df_sample.to_csv('/content/drive/My Drive/CS 5112 Final Project/Code/latest data/okcupid-cleaned-dropped-10k.csv')

In [67]:
df_sample

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,height,job,sign,smokes,arabic,chinese,english,german,hindi,italian,japanese,korean,portuguese,russian,spanish,turkish,has kids,wants kids,likes dogs,likes cats,new_religion
30518,26,single,f,straight,5,vegan,2,sometimes,graduated from college/university,white,62.0,artistic / musical / writer,whatever,no,no,no,yes,no,no,no,no,no,no,no,yes,no,no,yes,yes,yes,agnosticism
56256,38,single,f,straight,5,anything,2,never,working on grad,white,69.0,science / tech / engineering,whatever,no,no,no,yes,no,no,no,no,no,no,no,no,no,no,no,yes,no,none
31846,38,single,m,straight,5,anything,3,never,graduated from college/university,white,68.0,science / tech / engineering,whatever,no,no,no,yes,no,no,no,no,no,no,no,no,no,no,no,no,no,none
38659,38,single,f,straight,4,anything,0,never,graduated from college/university,asian,63.0,other,whatever,no,no,no,yes,no,no,no,no,no,no,no,no,no,no,no,no,no,catholicism
20309,24,single,f,gay,4,anything,2,sometimes,graduated from high school,white,68.0,retired,does_not_matter,no,no,no,yes,no,no,no,no,no,no,no,no,no,no,no,no,no,other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29717,28,single,m,straight,5,anything,1,sometimes,working on college/university,black,75.0,other,whatever,trying to quit,no,no,yes,no,no,no,no,no,no,no,no,no,no,no,no,no,christianity
32132,22,single,f,bisexual,5,vegetarian,2,sometimes,graduated from college/university,black,69.0,hospitality / travel,whatever,yes,no,no,yes,no,no,no,no,no,no,no,no,no,no,no,yes,no,none
8119,22,single,m,straight,5,anything,2,sometimes,working on college/university,white,72.0,law / legal services,does_not_matter,no,no,no,yes,no,no,no,no,no,no,no,no,no,no,no,yes,yes,agnosticism
7887,20,single,f,straight,2,other,1,never,working on college/university,mixed,71.0,sales / marketing / biz dev,does_not_matter,no,no,no,yes,no,no,no,no,no,no,no,no,no,no,yes,yes,no,christianity
